<a href="https://colab.research.google.com/github/hewanshrestha/Visual-Computing-Exercises/blob/master/FashionMNIST_with_Stacked_Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Loading Packages**

In [16]:
import tqdm
import torch
import numpy as np
import torchvision
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

**Loading Data**

In [17]:
transform = transforms.Compose([transforms.ToTensor()])
BatchSize = 1000

trainset = torchvision.datasets.FashionMNIST(root='./FMNIST', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BatchSize,
                                          shuffle=True, num_workers=4) # Creating dataloader

testset = torchvision.datasets.FashionMNIST(root='./FMNIST', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BatchSize,
                                         shuffle=False, num_workers=4) # Creating dataloader

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./FMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./FMNIST/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./FMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./FMNIST/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./FMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./FMNIST/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./FMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./FMNIST/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [19]:
# Check availability of GPU

use_gpu = torch.cuda.is_available()
if use_gpu:
    print('GPU is available!')
    device = "cuda"
else:
    print('GPU is not available!')
    device = "cpu"

GPU is available!


**Defining Autoencoder**

In [18]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28*28, 400),
            nn.Tanh())
        self.decoder = nn.Sequential(
            nn.Linear(400, 28*28),
            nn.Sigmoid())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


net = autoencoder()
print(net)

net = net.double().to(device)

autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=400, bias=True)
    (1): Tanh()
  )
  (decoder): Sequential(
    (0): Linear(in_features=400, out_features=784, bias=True)
    (1): Sigmoid()
  )
)


**Training Autoencoder**

In [20]:
epochs = 10
learning_rate = 0.98
criterion = nn.MSELoss()

for epoch in range(epochs):  # loop over the dataset multiple times
    runningLoss = 0.0
    for data in tqdm.tqdm_notebook(trainloader):
        # get the inputs
        inputs, labels = data
        inputs = inputs.view(-1, 28*28).double().to(device)
        net.zero_grad()  # zeroes the gradient buffers of all parameters
        outputs = net(inputs) # forward 
        loss = criterion(outputs, inputs) # calculate loss
        loss.backward() #  backpropagate the loss
        for f in net.parameters():
            f.data.sub_(f.grad.data * learning_rate) # weight = weight - learning_rate * gradient (Update Weights)
        runningLoss += loss.item()
        
    print('At Iteration : %d / %d  ;  Mean-Squared Error : %f'%(epoch + 1,epochs,
                                                                        runningLoss/(60000/BatchSize)))
print('Finished Training')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/60 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


At Iteration : 1 / 10  ;  Mean-Squared Error : 0.142281


  0%|          | 0/60 [00:00<?, ?it/s]

...

At Iteration : 8 / 10  ;  Mean-Squared Error : 0.067930


  0%|          | 0/60 [00:00<?, ?it/s]

At Iteration : 9 / 10  ;  Mean-Squared Error : 0.065141


  0%|          | 0/60 [00:00<?, ?it/s]

At Iteration : 10 / 10  ;  Mean-Squared Error : 0.062713
Finished Training


**Stacking Layers**

In [21]:
# Adding New Layer (Stacking)
net.encoder.add_module('New_Encoder_Layer', nn.Sequential(nn.Linear(400, 256),nn.Tanh()))
net.encoder.add_module('New_Decoder_Layer', nn.Sequential(nn.Linear(256, 400),nn.Tanh()))
print(net)
net = net.double().to(device)

autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=400, bias=True)
    (1): Tanh()
    (New_Encoder_Layer): Sequential(
      (0): Linear(in_features=400, out_features=256, bias=True)
      (1): Tanh()
    )
    (New_Decoder_Layer): Sequential(
      (0): Linear(in_features=256, out_features=400, bias=True)
      (1): Tanh()
    )
  )
  (decoder): Sequential(
    (0): Linear(in_features=400, out_features=784, bias=True)
    (1): Sigmoid()
  )
)


**Train Autoencoder**

In [22]:
for epoch in range(epochs):  # loop over the dataset multiple times
    runningLoss = 0.0
    for data in tqdm.tqdm_notebook(trainloader):
        # get the inputs
        inputs, labels = data
        inputs = inputs.view(-1, 28*28).double().to(device)
        net.zero_grad()  # zeroes the gradient buffers of all parameters
        outputs = net(inputs) # forward 
        loss = criterion(outputs, inputs) # calculate loss
        loss.backward() #  backpropagate the loss
        for f in net.parameters():
            f.data.sub_(f.grad.data * learning_rate) # weight = weight - learning_rate * gradient (Update Weights)
        runningLoss += loss.item()
        
    print('At Iteration : %d / %d  ;  Mean-Squared Error : %f'%(epoch + 1,epochs,
                                                                        runningLoss/(60000/BatchSize)))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/60 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


At Iteration : 1 / 10  ;  Mean-Squared Error : 0.088827


  0%|          | 0/60 [00:00<?, ?it/s]

...

At Iteration : 8 / 10  ;  Mean-Squared Error : 0.049700


  0%|          | 0/60 [00:00<?, ?it/s]

At Iteration : 9 / 10  ;  Mean-Squared Error : 0.048730


  0%|          | 0/60 [00:00<?, ?it/s]

At Iteration : 10 / 10  ;  Mean-Squared Error : 0.047965


**Modifying the autoencoder for classification**

In [23]:
# Removing the decoder module from the autoencoder
new_classifier = nn.Sequential(*list(net.children())[:-1])
net = new_classifier
new_classifier = nn.Sequential(*list(net[0].children())[:-1])
net = new_classifier
# Adding linear layer for 10-class classification problem
net.add_module('classifier', nn.Sequential(nn.Linear(256, 10),nn.LogSoftmax(dim=1)))
print(net)
net = net.double().to(device)

Sequential(
  (0): Linear(in_features=784, out_features=400, bias=True)
  (1): Tanh()
  (2): Sequential(
    (0): Linear(in_features=400, out_features=256, bias=True)
    (1): Tanh()
  )
  (classifier): Sequential(
    (0): Linear(in_features=256, out_features=10, bias=True)
    (1): LogSoftmax(dim=1)
  )
)


**Training Classifier**

In [28]:
epochs = 40
learning_rate = 0.1
criterion = nn.NLLLoss()

for epoch in range(epochs):  # loop over the dataset multiple times

    runningLoss = 0.0
    for data in tqdm.tqdm_notebook(trainloader):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.view(-1, 28*28).double().to(device), labels.to(device)
        net.zero_grad()  # zeroes the gradient buffers of all parameters
        outputs = net(inputs) # forward 
        loss = criterion(outputs, labels) # calculate loss
        loss.backward() #  backpropagate the loss
        for f in net.parameters():
            f.data.sub_(f.grad.data * learning_rate) # weight = weight - learning_rate * gradient (Update Weights)
        runningLoss += loss.item()
        correct = 0
        total = 0
    net.eval()
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            inputs, labels = inputs.view(-1, 28*28).double().to(device), labels.to(device)
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum()
    print('At Iteration : %d / %d  ;  Train Error : %f ;Test Accuracy : %f'%(epoch + 1,epochs,
                                                                        runningLoss/(60000/BatchSize),100 * correct /float(total)))
print('Finished Training')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/60 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


At Iteration : 1 / 40  ;  Train Error : 0.392298 ;Test Accuracy : 84.879997


  0%|          | 0/60 [00:00<?, ?it/s]

...

At Iteration : 38 / 40  ;  Train Error : 0.330981 ;Test Accuracy : 86.430000


  0%|          | 0/60 [00:00<?, ?it/s]

At Iteration : 39 / 40  ;  Train Error : 0.328295 ;Test Accuracy : 86.459999


  0%|          | 0/60 [00:00<?, ?it/s]

At Iteration : 40 / 40  ;  Train Error : 0.326200 ;Test Accuracy : 86.459999
Finished Training
